In [1]:
with open('input.txt', 'r', encoding='UTF-8') as fin:
    text = fin.read()

In [2]:
print(f"length of dataset: {len(text)}")

length of dataset: 1115394


In [3]:
print(text[:256])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:



In [4]:
# create "vocab"
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# character-level tokenization for the character-based transformer
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]           # encoder: take a string, output a list of int
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: take a list of int, output a string

print(encode("hey there"))
print(decode(encode("hey there")))

[46, 43, 63, 1, 58, 46, 43, 56, 43]
hey there


----

In [6]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:256])                                 # the 256 chars# we peeked at earlier

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
# train-validation split: 90-10
n = int(0.9*len(data))
train_data = data[:n]
valid_data = data[n:]

In [8]:
block_size = 8

train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
# context size will be allowed to range from 1 to block_size
# this will allow the model to make inferences of size ranging from 1 to block_size
x = train_data[:block_size]
y = train_data[1:block_size+1]                    # shift window 1 step into future!

for t in range(block_size):
    context = x[:t+1]                             # context up through the present t
    target  = y[t]                                # the char following position t
    print(f"when input is {context}, the target is: {target}")

when input is tensor([18]), the target is: 47
when input is tensor([18, 47]), the target is: 56
when input is tensor([18, 47, 56]), the target is: 57
when input is tensor([18, 47, 56, 57]), the target is: 58
when input is tensor([18, 47, 56, 57, 58]), the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


----

In [10]:
torch.manual_seed(1337)
batch_size = 4    # how many independent sequences will we process in parallel?
block_size = 8    # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else valid_data
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    return x, y


xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print()
print("targets:")
print(yb.shape)
print(yb)
print()

print("----")

# so we each batch contains 32 training examples (4 parallel seq. x 8 max. context)
for b in range(batch_size):        # batch dimension
    for t in range(block_size):    # time dimension
        context = xb[b, :t+1]
        target  = yb[b, t]
        print(f"when input is {context.tolist()}, the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

----
when input is [24], the target: 43
when input is [24, 43], the target: 58
when input is [24, 43, 58], the target: 5
when input is [24, 43, 58, 5], the target: 57
when input is [24, 43, 58, 5, 57], the target: 1
when input is [24, 43, 58, 5, 57, 1], the target: 46
when input is [24, 43, 58, 5, 57, 1, 46], the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43], the target: 39
when input is [44], the target: 53
when input is [44, 53], the target: 56
when input is [44, 53, 56], the target: 1
when input is [44, 53, 56, 1], the target: 58
when input is [44, 53, 56, 1, 58], the target: 46
when i

----

re: [`cross_entropy`](https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html), we need to reshape both `logits` and `targets`...

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        
        # each token directly reads off the logits
        # for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
 
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of ints
        logits = self.token_embedding_table(idx)    # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            
            # focus only on the last time step (bigram)
            logits = logits[:, -1, :] # becomes (B, c)
            
            # apply softmax to get probabilitis
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx    
            


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print()

# we kick of a test of generate
# by passing in a tensor with 0 (the newline char)
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [12]:
# from negative log-likelihood, we can quickly calculate 
# what our loss should be...

import math
-math.log(1/vocab_size)

# so... we are still not predicting correctly
# with this current model implementation...

4.174387269895637

----

In [13]:
# OK, the initial results of our Bi-gram model are dismal,
# so let's try actually training it and see if we can do 
# better
#
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [14]:
torch.manual_seed(1337)

# a typical training loop...
batch_size = 32

for steps in range(10000):
    
    # sample a batch of data...
    xb, yb = get_batch("train")
    
    # evaluate the loss...
    logits, loss = m(xb, yb)
    
    # zero-out the gradients from the previous step...
    optimizer.zero_grad(set_to_none=True)
    
    # get the gradients for all the parameters...
    loss.backward()
    
    # use those gradients to update our parameters!
    optimizer.step()

    #print(loss.item())
print(loss.item())

2.510111093521118


In [16]:
# now that we should have some improvement 
# via training our bi-gram model, let's see
# what the output looks like now...
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


ANA:

KI tus!
msetpecondall alll casove; t bre mit an. thanouig!

Ty at ldaysicome?
NTh haveo and ink! I sul Bed hernong! KRTI E:
F t d mome garesth oust he ovee perrear hanorerin gew,
RI t;
MEMe thous mace ffean ck trbeeds y hadowenarng pongox'siche s the nd cke fowe fo ES:
S iss, head be?

Surpl w
